so I will have data as ndarray at bottleneck as N by D
I will have the class labels acquired by kmeans
I want to find correspondance samples as
a) two random samples that are in cluster
b) some sample and the center sample

which wil be acquired as in and out indices by a function that takes nxd array and cluster labels

In [ ]:
import os, sys
import importlib as impL
import numpy as np
from pandas import DataFrame as pd_df

In [ ]:
def get_parent_dir(x):
    return os.path.dirname(x)
get_parent_dir(sys.path[0])

In [ ]:
sys.path.insert(1,os.path.join(get_parent_dir(sys.path[0]),'vae_torch'))
sys.path.insert(1,get_parent_dir(sys.path[0]))

In [ ]:
import vae_torch_model as vtm
import helperFuncs as funcH
desktop_dir = funcH.getVariableByComputerName('desktop_dir')

In [ ]:
def load_from_saved_corr_file(fn = '/home/doga/Desktop/correspondance_find.npz'):
    a = np.load(fn)
    predictions = a['predictions']
    bottleneck_vec = a['bottleneck_vec']
    labels = a['labels']
    print("loaded from file - ", fn)
    print(predictions.shape)
    print(bottleneck_vec.shape)
    print(labels.shape)
    return bottleneck_vec, predictions, labels

def analyze_corresondance_results(correspondance_tuple, centroid_df, pred_vec, lab_vec):
    if centroid_df is not None:
        df = pd_df({'labels': lab_vec[np.asarray(centroid_df['sampleID'], dtype=int)],
                    'klusterID': np.asarray(centroid_df['klusterID'], dtype=int),
                    'sampleCounts': np.asarray(centroid_df['num_of_samples'], dtype=int)})
        print('correspondance results({:}):'.format(len(correspondance_tuple[0])))
        print(df.groupby(['labels'])[['labels', 'sampleCounts']].sum())
    
    corr_in_clust = pred_vec[correspondance_tuple[0]]
    corr_ou_clust = pred_vec[correspondance_tuple[1]]
    _confMat_corr_preds = confusion_matrix(corr_in_clust, corr_ou_clust)
    acc_corr_preds = 100 * np.sum(np.diag(_confMat_corr_preds)) / np.sum(
        np.sum(_confMat_corr_preds))
    print("_confMat_corr_preds - acc({:6.4f})".format(acc_corr_preds))

    corr_in_labels = lab_vec[correspondance_tuple[0]]
    corr_ou_labels = lab_vec[correspondance_tuple[1]]
    _confMat_corr = confusion_matrix(corr_in_labels, corr_ou_labels)
    acc_corr = 100 * np.sum(np.diag(_confMat_corr)) / np.sum(np.sum(_confMat_corr))
    print("confMat - acc({:6.4f}), correspondance match:\n".format(acc_corr), pd_df(_confMat_corr))

b_v = np.random.rand(10,3)
#print(b_v)

In [ ]:
# np.savez('/home/doga/Desktop/correspondance_find.npz', bottleneck_vec=bottleneck_vec, predictions=pred_vec, labels=lab_vec)
bottleneck_vec, pred_vec, lab_vec = load_from_saved_corr_file(os.path.join(desktop_dir, 'correspondance_find_epoch1009_conf516.npz'))

In [ ]:
impL.reload(funcH)
def get_cluster_correspondance_ids_jupy(X, cluster_ids, correspondance_type="shuffle", verbose=0):
    # uses X to find the center sample
    # returns inds_in, inds_out where:
    # if correspondance_type=='shuffle'
    # inds_in : shuffled indices of a cluster
    # inds_out: shuffled indices of a cluster
    # elseif correspondance_type=='centered'
    # inds_in : some_sample_id
    # inds_out: the center of cluster of that sample_id
    centroid_df = funcH.get_cluster_centroids(ft=X, predClusters=cluster_ids, verbose=0)
    uq_pr = np.unique(cluster_ids)
    inds_in = []
    inds_out = []
    num_of_samples = []
    for i in range(len(uq_pr)):
        cluster_id = uq_pr[i]
        cluster_inds = funcH.getInds(cluster_ids, i)
        num_of_samples.append(len(cluster_inds))
        if correspondance_type == 'shuffle':
            iin_cur = cluster_inds.copy()
            np.random.shuffle(iin_cur)
            out_cur = cluster_inds.copy()
            np.random.shuffle(out_cur)
        elif 'knear' in correspondance_type:
            if verbose > 0:
                print("\n***\nknear-row{:}\n".format(i), cluster_inds)
            k = int(correspondance_type.replace('knear', ''))
            # look at the closest k samples for each sample
            X_sub = X[cluster_inds, :]
            k = np.minimum(len(cluster_inds), k)
            d_inds, d_vals = funcH.get_dist_inds(X_sub, k=k, metric="euclidean", verbose=0)
            # d_inds are from 0 to len(cluster_inds)
            # we want to switcth them with real cluster_inds
            if verbose > 2:
                print("cluster_inds:\n", cluster_inds)
                print("d_inds in:", d_inds)
            # d_inds.shape = [len(cluster_inds), k]
            # each row represents a sample and all columns represent its nearest neighbours
            # so i need to have each corr and k neighbours as correspondant frames
            sidx = np.array([cluster_inds, ] * k).T.flatten()
            if verbose > 1:
                print("i = ", i)
                print("sidx = \n", sidx)
            d_inds = cluster_inds[d_inds.flatten()]
            if verbose > 1:
                print("d_inds = \n", d_inds)
            iin_cur = sidx.copy()
            out_cur = d_inds.copy()
        else:
            center_sample_inds = centroid_df['sampleID'].iloc[i]
            if verbose > 0:
                print("cluster_id({:-3d}), sampleCount({:-4d}), centerSampleId({:-5d})".format(int(cluster_id),
                                                                                               len(cluster_inds),
                                                                                               center_sample_inds))
            # inds_in <--all sampleids except cluster center
            # inds_out<--cluster sample id with length of inds_in
            iin_cur = np.asarray(cluster_inds[np.where(center_sample_inds != cluster_inds)], dtype=int).squeeze()
            out_cur = np.asarray(np.ones(iin_cur.shape) * center_sample_inds, dtype=int)

        if verbose > 0:
            print("iin_cur.shape{:}, out_cur.shape{:}".format(iin_cur.shape, out_cur.shape))
            #if i == 0:
            print("iin=", iin_cur)
            print("out=", out_cur)
        inds_in.append(iin_cur)
        inds_out.append(out_cur)

    # first concatanate the lists into ndarray
    inds_in = np.asarray(np.concatenate(inds_in), dtype=int)
    inds_out = np.asarray(np.concatenate(inds_out), dtype=int)

    if True:  # 'knear' not in correspondance_type:
        # now a-b and b-a
        ii_ret = np.asarray(np.concatenate([inds_in, inds_out]), dtype=int)
        io_ret = np.asarray(np.concatenate([inds_out, inds_in]), dtype=int)
    else:
        ii_ret = inds_in.copy()
        io_ret = inds_out.copy()

    # now shuffle so that clusters not sorted in learning
    print("shuffle all")
    p = np.random.permutation(len(ii_ret))
    ii_ret = ii_ret[p]
    io_ret = io_ret[p]

    if verbose > 0:
        print("inds_in.shape{:}, inds_out.shape{:}".format(inds_in.shape, inds_out.shape))
    centroid_df['num_of_samples'] = num_of_samples
    return (ii_ret, io_ret), centroid_df

In [ ]:
correspondance_type = 'knear1'
impL.reload(funcH)
correspondance_tuple, centroid_df = get_cluster_correspondance_ids_jupy(X=bottleneck_vec, cluster_ids=pred_vec,
                                    correspondance_type=correspondance_type,
                                    verbose=0)

In [ ]:
from sklearn.metrics import confusion_matrix
analyze_corresondance_results(correspondance_tuple, centroid_df, pred_vec, lab_vec)

In [ ]:
impL.reload(funcH)
D, sort_inds = funcH.create_dist_mat(x=bottleneck_vec, metric="euclidean", verbose=0, use_less=True)

In [ ]:
D = np.float32(D)

In [ ]:
impL.reload(funcH)
dist_dict = funcH.get_linearized_distance_matrix(D,verbose=1, sort_dist="ascend")

In [ ]:
from sklearn.metrics import confusion_matrix
analyze_corresondance_results((corr_A,corr_B), centroid_df=None, pred_vec=pred_vec, lab_vec=lab_vec)

In [ ]:
impL.reload(funcH)
corr_inds, centroid_df = funcH.get_cluster_correspondance_ids(b_v, cluster_ids=[0,0,0,0,1,1,2,2,1,1], correspondance_type="knear4", verbose=0)
print(pd_df({"in":corr_inds[0],"out":corr_inds[1]}))
print(centroid_df)

In [ ]:
a,  centroid_df = get_cluster_correspondance_ids(bottleneck_vec, predictions, correspondance_type="shuffle")

In [ ]:
import vae_scripts as vs

In [ ]:
print(corr_A.shape)
print(corr_B.shape)

In [ ]:
corr_B = np.squeeze(sort_inds[0,:])
corr_A = np.arange(corr_B.size,dtype=int)

In [ ]:
vs.run_compare_list(experiments_folder="/home/doga/DataFolder/vaesae_experiments_cor",
                     data_log_keys=['tr_te', 'te'],
                     loss_key_list=[ 'bottleneck_kmeans', 'reconstruction'],
                     exp_base_name='exp_linear_vae_FMNIST_is28_cf',
                     ae_f_name_base='ae_ft_linear_vae_FMNIST_is28.npy')

In [ ]:
import vae_utils as vu
impL.reload(vu)
vu.plot_cf(cf_int=540, data_log_keys = ['tr_te'], max_act_ep=20, plot_cnt = 3,
            select_id_type='linspace', k_loss_disp_list={'bottleneck_kmeans'},
            experiments_folder=None,  # '/home/doga/DataFolder/vaesae_experiments/FM'
            exp_base_name=None,  # 'exp_conv_ae_simple_is28_cf'
            plt_min_max_lines=True)

In [ ]:
import vae_utils as vu
impL.reload(vu)
vu.plot_cf(cf_int=541, data_log_keys = ['tr_te'], max_act_ep=None, plot_cnt = 3,
            select_id_type='linspace', k_loss_disp_list={'bottleneck_kmeans'},
            experiments_folder=None,  # '/home/doga/DataFolder/vaesae_experiments/FM'
            exp_base_name=None,  # 'exp_conv_ae_simple_is28_cf'
            plt_min_max_lines=True)

In [ ]:
impL.reload(vu)
vu.plot_cf(cf_int=532, data_log_keys = ['tr_te', 'te'], max_act_ep=None, plot_cnt = 3,
            select_id_type='linspace', k_loss_disp_list={'bottleneck_kmeans'},
            experiments_folder=None,  # '/home/doga/DataFolder/vaesae_experiments_cor'
            exp_base_name=None,  # 'exp_linear_vae_FMNIST_is28_cf'
            plt_min_max_lines=True)

In [ ]:
silhouette_avg, silhouette_values_hl = funcH.calc_silhouette_params(bottleneck_vec, pred_vec)

In [ ]:
kluster_centroids = funcH.get_cluster_centroids(ft=bottleneck_vec, predClusters=pred_vec, kluster_centers=None, verbose=0)

In [ ]:
result_dict = funcH.analyze_silhouette_values(silhouette_values_hl, pred_vec, lab_vec,
                                    centroid_info_pdf=kluster_centroids,
                                    label_names=None, conf_plot_save_to='',
                                    figsize=(12, 5), lw=[4, 3, 2], show_title=False, str_deg=15, str_size=12)

1. we have silhouette scores in silhouette_values_hl
2. predictions and their ground truth labels at pred_vec and lab_vec
3. what if for each cluster we only take the knear1 samples,
   sort them by their sum of 2 silhouette values
   and only approve if the couple is above average silhouette score?
4. correspondance_tuple hass the original idx of values
   silhouette_values_hl also is that way

In [ ]:
print(correspondance_tuple)
n = len(pred_vec)
print(n)

In [ ]:
tuple_sihouette_score_sum = np.asarray([silhouette_values_hl[correspondance_tuple[0][i]]+silhouette_values_hl[correspondance_tuple[1][i]] for i in range(n)])

In [ ]:
tuple_idx = np.argsort(-tuple_sihouette_score_sum)

In [ ]:
print(tuple_sihouette_score_sum[tuple_idx[:10]])

In [ ]:
tup_sor_a_idx = correspondance_tuple[0][tuple_idx]
tup_sor_b_idx = correspondance_tuple[1][tuple_idx]
lab_vec_a = lab_vec[tup_sor_a_idx]
lab_vec_b = lab_vec[tup_sor_b_idx]
print(lab_vec_a)
print(lab_vec_b)
_cn_a = []
_cn_b = []
uniq_class_cnt_perc_a = np.zeros(lab_vec_a.shape)
uniq_class_cnt_perc_b = np.zeros(lab_vec_b.shape)
for i in range(n):
    if lab_vec_a[i] not in _cn_a:
        _cn_a.append(lab_vec_a[i])
    if lab_vec_b[i] not in _cn_b:
        _cn_b.append(lab_vec_b[i])
    uniq_class_cnt_perc_a[i] = len(_cn_a)
    uniq_class_cnt_perc_b[i] = len(_cn_b)
print(uniq_class_cnt_perc_a)
print(uniq_class_cnt_perc_b)

In [ ]:
print(np.sum(lab_vec_a==lab_vec_b)/len(lab_vec_a))

In [ ]:
pred_cumsum = np.cumsum(lab_vec_a == lab_vec_b) / np.cumsum(lab_vec_b==lab_vec_b)
max_run_acc_idx = np.argmax(pred_cumsum)
max_run_acc = pred_cumsum[max_run_acc_idx]
print("max_run_acc({:6.4f}), at {:d}(%{:4.2f})".format(max_run_acc, max_run_acc_idx, max_run_acc_idx/n))
data_perc_vec = np.arange(0, len(pred_cumsum)) / len(pred_cumsum)

In [ ]:
import matplotlib.pyplot as plt
plt.close('all')
fig, ax = plt.subplots(1, figsize=(12,8), dpi=180)
ax.plot(np.arange(max_run_acc_idx+5), pred_cumsum[:max_run_acc_idx+5], lw=2, label='accuracy', color='blue', ls='-', zorder=0)
ax.plot(np.arange(max_run_acc_idx+5), tuple_sihouette_score_sum[tuple_idx[:max_run_acc_idx+5]], lw=2, label='silhouette_prec', color='green', ls='-', zorder=0)
ax.plot(np.arange(max_run_acc_idx+5), uniq_class_cnt_perc_a[:max_run_acc_idx+5]/10.0, lw=1, label='silhouette_prec', color='purple', ls='-', zorder=0)
ax.plot(np.arange(max_run_acc_idx+5), uniq_class_cnt_perc_b[:max_run_acc_idx+5]/10.0, lw=1, label='silhouette_prec', color='cyan', ls='-', zorder=0)
plt.show()

plt.close('all')
fig, ax = plt.subplots(1, figsize=(12,8), dpi=180)
ax.plot(data_perc_vec, pred_cumsum, lw=2, label='accuracy', color='blue', ls='-', zorder=0)
ax.plot(data_perc_vec, tuple_sihouette_score_sum[tuple_idx], lw=2, label='silhouette_prec', color='green', ls='-', zorder=0)
ax.plot(data_perc_vec, uniq_class_cnt_perc_a/10.0, lw=3, label='silhouette_prec', color='purple', ls='-', zorder=0)
ax.plot(data_perc_vec, uniq_class_cnt_perc_b/10.0, lw=1, label='silhouette_prec', color='cyan', ls='-', zorder=0)
plt.show()

In [ ]:
pip install mplcursors

In [ ]:
import torch
import numpy as np
b_v = np.random.rand(4,1,10)
print(b_v.shape[0])
print(1 in b_v.shape)
print(b_v.squeeze().shape)
print(1 in b_v.squeeze().shape)
weights = np.random.rand(4,)
print(weights)
print(weights.shape)
torch.from_numpy(np.array([weights,]*10).T).size()